<img align="center" width="12%" style="padding-right:10px;" src="./Images/Ccom.png">

# Integrated Seabed Mapping Systems <a href="https://teams.microsoft.com/l/channel/19%3a6d420007fb504c57850639e54a52945b%40thread.tacv2/Lab%2520D?groupId=b7209537-725b-40fc-bdbe-212e9689ef18&tenantId=d6241893-512d-46dc-8d2b-be47e25f5666"><img src="./Images/help.png"  title="Ask questions on Teams" align="right" width="10%" ></a><br><br>  Lab D: 

___
## D0 Create Python Script - Load the Dependecies

We will run this lab a little different. Rather than adding and editing code in this Notebook we will use a script that we will run. This has the advantage that we ensure that all the code is executed sequentially. It also highlights another capability of Jupyter Notebooks.

Create a LabD.py File and load the dependencies 

    # D0 Create Python Script - Load the Dependecies

    import sys
    import import_ipynb
    import os.path
    import matplotlib as plt
    import numpy as np
    from datetime import datetime, timedelta
    from numpy import pi, arctan2, arccos, abs, sin, cos, tan, sqrt, sum, arctan
    from mycode.position import Position
    from mycode.twtt import TWTT
    from mycode.integration import Integration
    from mycode.integration import Motion
    from datetime import datetime, timezone
    from mycode.vessel import Vessel
    from mycode.ssp import SSP
    from mycode.ping import Ping
    from mycode.om_math import Rx, Ry, Rz



___
## D0.1 Add the Current Folder To the Path

Before we start in earnest we will have to do some more house keeping, we need to add the current folder to the path

    # D0.1 Add the Current Folder To the Path

    sys.path.append(os.getcwd())
    abs_path=os.path.abspath(os.path.curdir)
    print("Configured environment for Lab D")



## D0.2 Testing, Testing, D0.1, D0.2

In the code cell below we should see whether things worked



In [1]:
%load_ext autoreload
%autoreload 2

%run -i LabD.py

___
## D1 Defining the Vessel

<img align="center" width="80%" style="padding-right:10px;" src="./Images/conecone.jpg">

In Lab A we saw that to obtain a location and orientation of the sonars (from which to calculate their sonar-relative beam vectors) we need to utilize asynchronous observations of position and orientation. They are both available on the vessel, but at different locations or orientations. To relate the two together, we needed to establish a ship reference frame.

Within that reference frame we needed to establish the location and orientation of each sensor in turn. The net result is that we know the location of the transmit-transducer at the transmit epochs and the receive-transducer at the receive epochs.

The data presented here is from the same expedition as that presented in Lab A, i.e. the vessel data is the same
In the `LabD.py` script add the following to define the a `Vessel` object name `vessel`

    vessel = Vessel()
    vessel.metadata["name"]="USNS Henson"
    vessel.metadata["owned_by"]="United States Navy"
    vessel.metadata["operated_by"]="United States Navy"
    vessel.metadata["pos_source"]="NavCom (C-Nav)"
    vessel.metadata["sonar"]="Kongsberg EM710"
    vessel.metadata["mru"]="Applanix POS/MV 320"
    vessel.metadata["loa"]=100
    
Also add the lever arms and water line

    vessel.wl = -2.59
    vessel.lever_arm_trans = np.asarray([16.26, -1.75, 4.15]).reshape((3, 1))
    vessel.lever_arm_rec = np.array([14.82, -2.01, 4.17]).reshape((3, 1))
    vessel.lever_arm_pos = np.array([-5.73, -0.12, -30.00]).reshape((3, 1))
    vessel.lever_arm_mru = np.array([0, 0, 0]).reshape((3, 1))


In [2]:
%run -i LabD.py

print(vessel)

NameError: name 'vessel' is not defined

___
## D2 Representing the Transmit and Receive Array

For the directions it is helpful to represent the transducer arrays by unit vectors. Unit vectors have the unique property that the elements are the direction cosines with respect to the axis. We could proof this formally, but 
we will illustrate this at the hand of an example: the vector (1,1) has an angle of $1/4 \pi$ radians and a length of $\sqrt{2}$ Thus if we normalize the vector we end up with a vector $\left(\sqrt{2},\sqrt{2}\right)$ which has norm $1$. Similarly the $\cos(1/4 \pi) = \sqrt{2}$. 

Thus it is very convenient to represent direction using unit vectors, the elements are direct measures of the angle that the vector makes with the coordinate axes. i.e. the first element represents the cosine of the angle of the vector with the first coordinate axis, the 2nd with the 2nd axis, etc.

We will chose some directions of convenience - ideally the transmit transducer is aligned to the X-axis of a vessel thus a logical vector to represent its orientation is (1,0,0), ( the angle with the x-axis is zero, thus the first element is cos(0) = 1, the angle with the y- and z-axes is 90 degrees cos(90) = 0 thus our vector is (1, 0, 0)

We will represent the ideal transmit and receive transducers with the variable `tx_ideal` and `rx_ideal` respectively

In the `LabD.py` script add and complete the following 

    tx_ideal = np.array([[1],[0],[0]])
    rx_ideal = np.array([...])


In [ ]:
%run -i LabD.py

print( tx_ideal)
print( rx_ideal)

    Configured environment for Lab D
    [[1]
     [0]
     [0]]
    [[0]
     [1]
     [0]]


___
## D2.0 Representing the Transmit and Receive Array - Reality is Not Ideal

Unfortunately reality is not ideal. The transducer will not be mounted quite orthogonal and not quite aligned to the vessel reference frame. We will have to deal with some mount angles that are deviations from the ideal vectors. In the case of the 'Henderson' these are known, so we may take them into account.

In the `LabD.py` script add the following: 

    ma_tx=np.array([0.127,1.024,359.957-360])*pi/180  # Tx Mount angles 
    ma_rx=np.array([0.101,0.894,0.065])*pi/180    # Rx Mount angles 



In [ ]:
%run -i LabD.py

print( ma_tx)
print( ma_rx)

    Configered environment for Lab D
    [ 0.00221657  0.01787217 -0.00075049]
    [0.00176278 0.01560324 0.00113446]

___
## D3 Parameters for Beam Forming

In our case we will be reading data from a datagram collected with the Kongsberg system - all the data associated to the specific beam for the specific ping that we are interested in is contained in the data file `data_ping_4170.txt` that you may find in your `Data` folder.

In our particular case you will be analyzing beam 391, we will keep track of that here so that you may later on extract the right information for the beam depression angle. We will also give you the sound speed at the transducer here, as you should not use the sound speed from the sound speed profile for that - that would create a significant error in your ray tracing.

In the `LabD.py` script add the following:
    ss_tx=1543.7                       
    beam_select=391   

In [ ]:
%run -i LabD.py

print( ss_tx)
print( beam_select)

    Configured environment for Lab D
    1543.7
    391


___
## D4 Get the Positioning Data

We will get the positioning data and store them in a `Position` object called `positions`. We will do this in the same fashion as in Lab A - the data file is called `Lab_A_GNSS.txt` is located in the Data folder, should be read with the `read_jhc_file` method:

In the `LabD.py` script add and complete the following:
    positions = Position()
    positions.read_jhc_file(abs_path+...)

In [ ]:
%run -i LabD.py

print( positions)

    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    geodetic_units: rad
    height_units: m
    proj_units: m
    geoid_name: EGM08
    ellipsoid_name: WGS84
    height_relative_to: geoid
    time_basis: UTC
    proj_str: None
    Minimum latitude       : 0.278496
    Maximum latitude       : 0.278564
    Minimum longitude      : 2.548137
    Maximum longitude      : 2.548780
    Minimum height         : 27.18m
    Maximum height         : 82.38m
    Start Time             : 2011-05-08 03:49:11+00:00
    End Time               : 2011-05-08 03:49:11+00:00



___
## D4.0 Squaring the Positioning Data Away

As we saw in lab A we want to do our calculations in a Cartesian world if we can. Thus, just like in that lab, we want to transform our coordinates to UTM coordinates. For this we will use the `Position` class `carto_project` method

In the `LabD.py` script add the following:

    positions.carto_project("UTM", "ortho")

In [ ]:
%run -i LabD.py

print( positions.proj_pos)

    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    [[3.92719337e+05 3.92724827e+05 3.92730791e+05 ... 3.92723976e+05
      3.92718461e+05 3.92713022e+05]
     [1.76481175e+06 1.76481199e+06 1.76481229e+06 ... 1.76440513e+06
      1.76440523e+06 1.76440540e+06]
     [2.88950000e+01 2.91460000e+01 2.89800000e+01 ... 2.88460000e+01
      2.87760000e+01 8.23750000e+01]]


___
## D5 Get the Motion Data

We will get the motion data and store them in a `Motion` object called `motions`. Just like the positioning we will do this in the same fashion as in Lab A - the data file is called `Lab_A_MRU.txt` is located in the Data folder and should be read with the `read_jhc_file` method:

In the `LabD.py` script add and complete the following:

    motions = Motion()
    motions.read_jhc_file(...)

In [ ]:
%run -i LabD.py

print(motions)

    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Start time: 2011-05-08 03:49:10.572000+00:00
    End time:   2011-05-08 04:12:27.053000+00:00
    angle__units: rad
    distance_units: m
    time_basis: UTC
    Source File: /home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt



___
## D6 Get the Sound Speed Data

We will get the sound speed data and store them in a `SSP` object called `ssp`. This time the data file is called `Lab_A_SVP.txt` and is located in the Data folder. You should read it with the `read_jhc_file` method which you may add to the SSP class.

    def read_jhc_file(self, fullpath):
        # Check to see whether data already exists in the object
        
        if self.obs_depths:
            raise RuntimeError('SSP object already contains a profile')
            
        # Check the File's existence
        if os.path.exists(fullpath):
            self.metadata["Source File"] = fullpath
            print('Opening sound speed profile data file:' + fullpath)
        else:  # Raise a meaningful error
            raise RuntimeError('Unable to locate the input file' + fullpath)

        # Open, read and close the file
        svp_file = open(fullpath)
        svp_content = svp_file.read()
        svp_file.close

        # Tokenize the contents
        svp_lines = svp_content.splitlines()
        self.obs_time = datetime.fromtimestamp(
            float(svp_lines[1].split()[0]), timezone.utc)
        self.log_time = datetime.fromtimestamp(
            float(svp_lines[2].split()[0]), timezone.utc)
        self.obs_latitude = float(svp_lines[3].split()[0])
        self.obs_longitude = float(svp_lines[3].split()[1])
        self.vessel_latitude = float(svp_lines[4].split()[0])
        self.vessel_longitude = float(svp_lines[4].split()[1])
        self.metadata["count"] = int(svp_lines[5].split()[0])

        count = 0  # initialize the counter for the number of rows read

        for svp_line in svp_lines[16:]:
            observations = svp_line.split()  # Tokenize the stringS
            self.obs_sample.append(float(observations[0]))
            self.obs_depths.append(float(observations[1]))
            self.obs_ss.append(float(observations[2]))
            count += 1

        if self.metadata["count"] != count:
            raise RuntimeError('Nr of Samples read ('+str(count) +
                               ') does not match metadata count (' +
                               str(self.metadata["count"])+')')

        # Process the data - in the jhc data files this is already a one-way profile,
        # this just for illustration
        array_ss = np.zeros((count, 3))

        # Sort the data samples by depth
        sorted_ss = sorted(zip(self.obs_depths, self.obs_ss))
        
        layer = 0
        for d, ss in sorted_ss:
            array_ss[[layer], [0]] = d
            array_ss[[layer], [1]] = ss
            layer += 1

        # Identify all the depths for which there are multiple observations
        mask = np.full((count, 1), True)
        mask[1:, [0]] = np.diff(array_ss[:, [0]], axis=0) != 0
        
        # Remove the duplicates - You really should get statistical representations here
        # but to keep this short just remove the duplicates
        array_ss = array_ss[mask[:, 0], ...]
        
        # Determine the gradients - Note the indexing: the gradient of the first layer 
        # is contained at the same index as the data for the TOP of the layer.
        array_ss[0:-1, [2]] = np.diff(array_ss[:, [1]],
                                          axis=0)/np.diff(array_ss[:, [0]], axis=0)

        # Estimate gradient for last layer assuming that the temperature and salinity remain the same
        # gradient solely a function of pressure (depth)
        array_ss[-1, [2]] = 0.017

        # Extend to 12000 m if necesarry - this is to get around some manufcturers requirements
        if self.obs_depths[-1] < 12000:
            ss = array_ss[-1:, [1]] + array_ss[-1:, [2]] \
             * (12000-array_ss[-1:, [0]])
            array_ss = np.vstack((array_ss, [12000, ss, 0.017]))
            
        # Make sure that the last gradient is 0.017
        array_ss[-1,2] = 0.017

        # Extend to 0 m if necesarry - assume well mixed
        if self.obs_depths[0] > 0:
            array_ss = np.vstack(
                ([0, array_ss[0, [1]], 0.], array_ss))
            
        # Step 5 Create a look-up array of twtts for each full layer
        # Allows for great gain in efficiency (do not have to calculate for each ping)
        self.twtt_layer = np.zeros((count, 1))
        
        for layer in range(0,self.metadata["count"]-1):
            if array_ss[layer, [2]] == 0:
                self.twtt_layer[layer] = 2 * \
                    (array_ss[layer+1, [0]] - array_ss[layer, [0]])/ \
                     array_ss[layer, [1]]
            else:
                self.twtt_layer[layer] = 2 / array_ss[layer, [2]] * \
                 log(array_ss[layer+1, [1]]/array_ss[layer, [1]])
        
        self.proc_ss = array_ss[:,1]
        self.proc_depth = array_ss[:,0]
        self.g = np.diff(self.proc_ss)/np.diff(self.proc_depth)
        self.g[self.g == 0] = .24
        
        # Updating the Profile
        for i in range(1,len(self.g)):
            self.proc_ss[i]=self.proc_ss[i-1]+(self.proc_depth[i] - self.proc_depth[i-1])*self.g[i-1]


In the `LabD.py` script add and complete the following:

    ssp = ...
    ssp.read_jhc_file(...)

In [ ]:
%run -i LabD.py

print(ssp.proc_depth)

    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    [0.0e+00 2.0e+00 3.0e+00 4.0e+00 5.0e+00 6.0e+00 7.0e+00 8.0e+00 9.0e+00
     1.0e+01 1.1e+01 1.2e+01 1.3e+01 1.4e+01 1.5e+01 1.6e+01 1.7e+01 1.8e+01
     1.9e+01 2.0e+01 2.1e+01 2.2e+01 2.3e+01 2.4e+01 2.5e+01 2.6e+01 2.7e+01
     2.8e+01 2.9e+01 3.0e+01 3.1e+01 3.2e+01 3.3e+01 3.4e+01 3.5e+01 3.6e+01
     3.7e+01 3.8e+01 3.9e+01 4.0e+01 4.1e+01 4.2e+01 4.3e+01 4.4e+01 4.5e+01
     4.6e+01 4.7e+01 4.8e+01 4.9e+01 5.0e+01 5.1e+01 5.2e+01 5.3e+01 5.4e+01
     5.5e+01 5.6e+01 5.7e+01 5.8e+01 5.9e+01 6.0e+01 6.1e+01 6.2e+01 6.3e+01
     6.4e+01 6.5e+01 6.6e+01 6.7e+01 6.8e+01 6.9e+01 7.0e+01 7.1e+01 7.2e+01
     7.3e+01 7.4e+01 7.5e+01 7.6e+01 7.7e+01 7.8e+01 7.9e+01 8.0e+01 8.1e+01
     8.2e+01 8.3e+01 8.4e+01 8.5e+01 8.6e+01 8.7e+01 8.8e+01 8.9e+01 9.0e+01
     9.1e+01 9.2e+01 9.3e+01 9.4e+01 9.5e+01 9.6e+01 9.7e+01 9.8e+01 9.9e+01
     1.2e+04]

___
## D7 Get the Ping Data

<img align="center" width="80%" style="padding-right:10px;" src="./Images/ping.png">

A single file *data_ping_4170.txt* is supplied which, together with the orientation and offsets will allow you to do the full calculation.

What information do you have on the transmit sector for the beam you chose?
From the data telegram, there is lots of other information that is provided about each sector. The only things you need are:

- the transmit steering angle and
- the time delay between the first sector firing and the one you have chosen.

Your actual time of transmission is the nominal time of the swath PLUS the extra delay while it waits for the other sectors to fire. For example, Sector 1 fired first (at the specified time). Sector 0 fires 0.4ms later and Sector 2 fire another 0.4ms after that (0.8ms w.r.t the central sector).

To save you some time a `Ping` class has been created for you - you may find it in your `mycode` folder and it is already included in the dependencies.


In the `LabD.py` script add and complete the following:

    ping = Ping()
    ping.read(...)

In [ ]:
%run -i LabD.py

print(ping)

___
## D7.0 Indexing the Ping Data

As you can see from the output above the data comes with a header that describes the properties of three sectors. In the figure above you can see that this sector data is followed by a table of beam specific data. One thing to realize is that the beam number is associated to a specific beam, not the line number of the data in the table. To clarifu it may be that beams 1,2,4 and, 5 were processed successfully, but beam 3 failed. The data for beam 3 is then not included in the table. This has a result that the data for beams 1 and 2 are in records 1 and 2, but the data for beams 4 and 5 are in records 3 and 4 respectively. 

Thus we need a mechanism to identify the proper record, fortunately the `Ping` class comes with the `get_beam_index` method that allows us to find the record index associated to our selected beam `beam_select`

In the LabD.py script add the following:

    i = ping.get_beam_index(beam_select)


In [ ]:
%run -i LabD.py

print(i)

    391


To use analyze beam data we may now use the index that we found, which in this case is the same as the beam number itself. For example, we may interested in what the sector time delay is for the beam of interest, we may find this using: 

    ping.tx_t_offset_beam[i]
    
Which will result in a `timedelta` 

In [ ]:
print(ping.tx_t_offset_beam[i])

    0:00:00.000800

___
## D8 Timing it Right

<img align="center" width="80%" style="padding-right:10px;" src="./Images/timing.bmp">

As you saw in Lab A all the integration is done based on time. We will need to know how the transducer are oriented at both the time of transmit and reception. Thus the first thing that we need to so is to determine what those points in time are. The ping time is contained in the variable `ping.tx_time`

In the `LabD.py` script add the following:

    t_tx = ping.tx_time+ping.tx_t_offset_beam[i]
    t_rx = t_tx+timedelta(0,ping.twtt[i])

What the code above does is to determine the time `t_tx` associated to the transmit, `ping.tx_time` is a `datetime` object and `ping.tx_t_offset_beam[i]` is a `timedelta` object, which allows us to handle the time more easily (I strongly suggest that you look at the `Ping` class implementation to gather greater understanding.

The next line then adds the TWTT for the beam to the transmit time to determine the reception time `t_rx`. However, `ping.twtt[i]` is a simple float and thus needs to be transformed to a `timedelta` before we may add it to `t_tx`.





In [ ]:
%run -i LabD.py

print(t_tx)
print(t_rx)

    2011-05-08 04:01:06.555800+00:00
    2011-05-08 04:01:06.785567+00:00

___
## D9 Getting the Motion Data for our Epochs

Now that we have the time we can orient the transducers at the time of transmit and reception. Unlike in lab A we only need to do this for a single set of transmit and receive epochs, however the same methods may be used. In our case we will add a few methods to the `Motion` class that will allow us to gather the right information



___
## D9.0 Getting Motion Vectors

The first thing that we are interested in is the orientation of the vessel at specific epochs - we will add the method
`get_motion` to the `Motion` class. This method will take a `datetime` argument an return the motion as a one dimensional `numpy` array. The order will be the rotation around the x-axis, y-axis, z-axis and the heave.

The implementation is really the same process that we used in the `Integration` class of Lab A.


**NOTE: IF YOU TOOK ESCI 872 YOU WILL ALREADY HAVE THIS METHOD**

    def get_motion(self, time = datetime.fromtimestamp(0, timezone.utc)):

        # Allocate Memory
        attitude = np.zeros(4)
        
        # 7.10.2 Map the Input Times to POSIX Times
        times = np.array([e.timestamp() for e in self.times])
        
        attitude[0] = np.interp(time.timestamp(), times, self.roll)
        attitude[1] = np.interp(time.timestamp(), times, self.pitch)
        attitude[2] = np.interp(time.timestamp(), times, self.yaw)
        attitude[3] = np.interp(time.timestamp(), times, self.heave)


In the LabD.py script add and complete the following:

    att_tx = motions.get_motion(t_tx)
    att_rx = motions.get_motion(...)

In [ ]:
%run -i LabD.py

print(att_tx)
print(att_rx)

    [ -0.0188  0.0365  3.7564  0.0378]
    [ -0.0277  0.0356  3.7634  0.1400]


___
## D9.1 Getting Orientation Matrices

For many of our purposes it makes more sense to use a rotation matrix to represent our orientation and leave out the heave, which is a translational motion. For this you should add the following method to your `Motion` class

    def get_rotation_matrix(self, time = datetime.fromtimestamp(0, timezone.utc)):
        # 7.11.1 Determining the Attitude
        att = self.get_motion(time)

        Rx = np.array([[1, 0,            0          ],
                       [0, cos(att[0]), -sin(att[0])],
                       [0, sin(att[0]),  cos(att[0])]])

        Ry = np.array([[ cos(att[1]),  0, sin(att[1])],
                       [ 0          ,  1, 0          ],
                       [-sin(att[1]),  0, cos(att[1])]])

        Rz = np.array([[cos(att[2]), -sin(att[2]), 0],
                       [sin(att[2]),  cos(att[2]), 0],
                       [0          ,  0          , 1]])
                       
        return Rz@Ry@Rx


In the LabD.py script add and complete the following:

    R_tx = motions.get_rot_matrix(t_tx)
    R_rx = motions....()

In [ ]:
%run -i LabD.py

print(R_tx)
print(R_rx)

    [[ -0.8163  0.5773  -0.0190]
     [ -0.5764  -0.8163  -0.0364]
     [ -0.0365  -0.0188  0.9992]]
    [[ -0.8123  0.5831  -0.0128]
     [ -0.5821  -0.8120  -0.0432]
     [ -0.0356  -0.0276  0.9990]]


___
## D10 Finding out Where We Are 

Next up is the determination of the vessel location at our epochs of interest - we will add the method
`get_position` to the `Position` class. This method will take a `datetime` argument an return the position as a one dimensional `numpy` array. The order of the elements should be the same as that in the `Position.proj_pos` array.

    def get_position(self, time = datetime.fromtimestamp(0, timezone.utc)):
        pos = np.zeros(3)
        times = np.array([e.timestamp() for e in self.times])
        pos[0] = np.interp(time.timestamp(), times, self.proj_pos[0])
        pos[1] = ...
        pos[2] = ...
        
        return pos
        
In the LabD.py script add and complete the following:

    geo_tx_ant = positions.get_position(t_tx)
    geo_rx_ant = positions.get_position(t_rx)
    
This gives the georeferenced position of the GNSS antenna at the transmit epoch `geo_tx_ant` and receive epoch `geo_rx_ant`.  

In [ ]:
%run -i LabD.py

print(geo_tx_ant)
print(geo_rx_ant)

    [3.96628073e+05 1.76453295e+06 2.85175464e+01]
    [3.96627798e+05 1.76453196e+06 2.84045010e+01]


___
## D11 Georeferencing the Lever Arms

Just like in Lab A we will need to determine our lever arms in georeferenced space. We will need to geo-reference the antenna lever arm to be able to position the reference point (rp) at both the transmit and receive epochs. Similarly we will need the georeferenced transducer lever arms to be able to locate the transducer using the rp in geo-referenced space. We may use the `Motion.geo_reference_la` method for this purpose.

First we will calculate the georeferenced GNSS lever arm `geo_tx_ant` for epoch `t_tx` and then `geo_tx_ant` for epoch `t_rx`

In the LabD.py script add and complete the following:

    geo_pos_la_tx = motions.geo_reference_la(t_tx, vessel.lever_arm_pos)
    geo_pos_la_rx = ...




In [ ]:
%run -i LabD.py

print(geo_pos_la_tx)
print(geo_pos_la_rx)

    [[ 5.1770]
     [ 4.4920]
     [ -29.7635]]
    [[ 4.9700]
     [ 4.7300]
     [ -29.7619]]


___
## D12 Calculating the Reference Position RP

Again like lab A we need to determine the location of our reference point `geo_pos_rp_tx` at `t_tx` and `geo_pos_rp_rx` at `t_rx`. We will store these locations in 3x1 column vectors. Just like in Lab A you have to be careful to be careful about axis alignment:

In the LabD.py script add and complete the following:

    geo_pos_rp_tx = np.zeros((3,1))
    geo_pos_rp_rx = ...

    geo_pos_rp_tx[0] =  geo_tx_ant[0] - geo_pos_la_tx[1]
    geo_pos_rp_tx[1] =  geo_tx_ant[1] - geo_pos_la_tx[0]
    geo_pos_rp_tx[2] = -geo_tx_ant[2] - geo_pos_la_tx[2]

    geo_pos_rp_rx[0] =  ...
    geo_pos_rp_rx[1] =  ...
    geo_pos_rp_rx[2] =  ...


In [ ]:
%run -i LabD.py

print(geo_pos_rp_tx)
print(geo_pos_rp_rx)

    [[ 396623.5810]
     [ 1764527.7769]
     [ 1.2460]]
    [[ 396623.0679]
     [ 1764526.9894]
     [ 1.3574]]


----
## D13 The Separation of Shot and Receive

<img align="center" width="80%" style="padding-right:10px;" src="./Images/separation_shot_receive.jpg">

As there is a finite time between transmission and reception, both the position and the orientation of the array will change in that interval. That interval can typically range from ~1/10 of second (<75m ranges) to 10’s of seconds for deep water systems.

If a single location value is to be used, the average of the transmitter position at transmit and receiver position at the average received TWTT can reasonably be used, as we did in Lab A. Strictly though, over the reception cycle, the receiver does propagate forward. But at typical vessel speeds (~4m/s) the distance traveled, compared to the one way sound speed (750 m/s) is a very small fraction (0.5%) which is smaller than the beam footprint. For example, the projected beam width of a 0.5$^{\circ}$ beam is 0.87% of Z (strictly slant range). Both these numbers are small compared to IHO Special Order horizontal accuracy requirements of 2m in 40m of water which corresponds to only 5% of Z.

**Note** that this excludes the physical separation of the transmitter and receiver acoustic centers (normally at least ½ the physical array length). If bottom detection is attempted in the acoustic near field, that separation may be significantly larger than the forward propagation of the sonar over the ping cycle.

Unlike lab A we cannot take a simple average, the beam steering is applied at the specific transmit and receive times, so we will create a virtual transducer with the transmit element located and oriented at `t_tx` and the receive element located and oriented at `t_rx`. Thus we need to have a vector that describes the distance traveled between those two epochs. We will use the vector `geo_pos_diff` for that purpose.

<img align="center" width="80%" style="padding-right:10px;" src="./Images/geo_pos_diff.PNG">

In the LabD.py script add the following:

    geo_pos_diff = geo_pos_rp_rx - geo_pos_rp_tx

In [2]:
%run -i LabD.py

print(geo_pos_diff)

NameError: name 'geo_pos_diff' is not defined

    Configured environment for Lab D
    Opening GNSS data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_GNSS.txt
    Opening motion data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_MRU.txt
    Opening sound speed profile data file:/home/jupyter-semmed/ESCI_OE_874/Data/Lab_A_SVP.txt
    Data/data_ping_4170.txt
    Opening sound speed profile data file:Data/data_ping_4170.txt
    [[ -0.7875]
     [ -0.5131]
     [ 0.1114]]



----
## D14 Course Over Ground

<img align="center" width="80%" style="padding-right:10px;" src="./Images/cog.PNG">

Now that we have this vector we know what the course over ground (cog) between the transmit and receive epoch was.

Note that after this it is more convenient to change the order in the `geo_pos_diff` vector

In the LabD.py script add the following:

    cog = arctan2(geo_pos_diff[0],geo_pos_diff[1])
    
    temp = geo_pos_diff.copy()
    geo_pos_diff[0] = temp[1]
    geo_pos_diff[1] = temp[0]

In [1]:
%run -i LabD.py

print(cog)
print(geo_pos_diff)

NameError: name 'cog' is not defined

    [ -2.5641]
    [[ -0.5131]
     [ -0.7875]
     [ 0.1114]]


----
## D15 Drift Angle

<img align="center" width="80%" style="padding-right:10px;" src="./Images/da.PNG">

The difference between the cog and the heading is known as the drift angle

Pet peeve alert (ignore if you like): often the drift angle is incorrectly referred to as the crab angle. The crab angle is actually a course correction applied by the navigator to achieve the desired course over ground! the Crab angle should therefore be in the opposite direction of the drift angle, but may be of a different magnitude.

remember that the heading at transmit is given by `att_tx[2]`

In the LabD.py script add and complete the following:

    da=(...-...[2])[0]
    while da < -pi:
         da = da + 2 * pi
    while da > pi:
        d...

In [ ]:
%run -i LabD.py

print(da)

    -0.037315440834639446

----
## D16 Heading Change

<img align="center" width="80%" style="padding-right:10px;" src="./Images/dh_1.PNG">

The next thing we need is to determine the heading change between the epochs, as this will contribute to the difference in alignment between the transmit transducer at `t_tx` and the receive transducer at `t_rx`.

<img align="center" width="80%" style="padding-right:10px;" src="./Images/dh_2.PNG">

In the LabD.py script add and complete the following:

    d_h=att_rx[2]-att_tx[2]

In [ ]:
%run -i LabD.py

print(d_h)

    0.006934786545932159


---
## D17 Equivalent Modeled Virtual Array

<img align="center" width="80%" style="padding-right:10px;" src="./Images/eq_model.jpg">

To calculate a single beam vector for the outgoing and incoming ray path, an equivalent array geometry needs to be defined in which the acoustic centers of the transmit and receive transducers are co-located.

The figure above illustrates the modeled beam vector solution. A virtual array location, half way in X,Y and Z from the acoustic center at transmit and receive is calculated. The alignment of the transmit transducer is defined by the attitude at transmit, the alignment of the receive transducer is determined by the attitude at reception. The arrays are no longer mutually orthogonal (even if the physically constructed arrays actually are)as time has elapsed.

We will create a new reference system based on the location and orientation of the transmit transducer at the transmit epoch.



## Creating A new Reference System

With the results from D14-D17 we can calculate the orientation of the long axis of the transmitter and receiver at the transmit and receive epochs. Recall from class that the steered beams are oriented around those long axes.

Ideally the transducers are aligned to the X and Y-axes and are orthogonal to each other at the time of transmission and reception. As we have seen this is not the case, but we have all the information needed to construct a virtual array. We know the orientation of the transmit array at transmit epoch `t_tx` and receive epoch `t_rx`. We also know how the vessel moved by in the intervening epoch `geo_pos_diff`. If we assume that the transmit transducer is aligned to the x-axis  then we need to determine where the receive transducer is relative to the transmit transducer. 

We will create a new reference system that is:

- Right handed Cartesian
- Uses SI units for scale
- Has the origin defined by rp location at the transmit 
- The ideal transducer long axis is aligned to the VRF x-axis 
- Has the 1-axis in the direction of the ideal long axis of the transmit transducer

----
## D17.0 Aligning `geo_pos_diff` to the 1-Axis

<img align="center" width="80%" style="padding-right:10px;" src="./Images/ref_pos_diff.PNG">

We want to create a new 1-axis that is aligned to the X-axis of the vessel reference frame at the time of transmit i.e., the new 1-axis will be in the direction of the heading at the time of transmit. This means that the lever arms at the time of transmit are already aligned to the direction of the new 1-axis. The vector `geo_pos_diff` connecting the RP at transmit and receive will need to be rotated. In the figure you will see that the resulting vector will be almost coincident with the 1-axis, differing by just the drift angle `da`. We will call the resulting vector `ref_pos_diff`.

In the LabD.py script add the following:

    ref_pos_diff = Rz(-att_tx[2])@geo_pos_diff
    
Note the value of the 2-axis element - it will typically be close to zero, unless a significant drift angle is encountered.

In [ ]:
%run -i LabD.py

print(ref_pos_diff)

    [[ 0.9393]
     [ -0.0351]
     [ 0.1114]]


----
## D17.1 New Transmit Transducer Lever Arm

<img align="center" width="80%" style="padding-right:10px;" src="./Images/ref_pos_diff.PNG">

The lever arm to the transmit transducer is already centered on the new origin (the RP at transmit) and aligned to the new 1-axis. Thus the only thing that we need to do to get the coordinates of the transmit transducer is to compensate for motion by rotating the lever arm for roll and pitch.

In the LabD.py script add and complete the following:

    # D17.1 New Transmit Transducer Lever Arm
    new_la_tx = Ry(...)@Rx(...)@vessel.lever_arm_trans

In [ ]:
%run -i LabD.py

print(new_la_tx)

    [[16.40170235]
     [-1.67178928]
     [ 3.5863464 ]]


----
## D17.2 New Receive Transducer Lever Arm

The lever arm to the receive transducer is not centered on the new origin (the RP at transmit) nor aligned to the new 1-axis. 

<img align="center" width="80%" style="padding-right:10px;" src="./Images/new_la_rx.PNG">

From the figure above we can see that the RP at reception is offset by the translation vector `ref_pos_diff`. We can also see that the vessel reference frame at the time of reception needs to be rotated by the difference in heading `d_h` between the transmit and receive epochs.

Finally the receive transducer lever arm will need to be rotated to compensate for roll and pitch at the reception epoch which is contained in the array `att_rx`.

Note that the order matters, we will rotate in the order roll, pitch, heading difference, i.e., $\text{R}_z\cdot\text{R}_y\cdot\text{R}_x$

In the LabD.py script add and complete the following:

    # D17.2 New Receive Transducer Lever Arm
    new_la_rx = Rz(...)@Ry(...)@Rx(...)@vessel.lever_arm_rec+ref_pos_diff
    
This positions the transducer with respect to the vessel reference frame as it was at the transmit epoch.

In [ ]:
%run -i LabD.py

print(new_la_rx)

    [[15.91315915]
     [-1.82514732]
     [ 3.8045833 ]]


----
## D17.3 Virtual Transducer location: Why colocation?

We are concerned with establishing the point on the seafloor at which the conical maximum response axes of the transmitter and receiver intersect. In reality, the two cones will be distorted by refraction. If the two acoustic centers are not at the same location, then the refracted ray paths from each center to the strike point will not be identical. However, if they are concentric, they will share the same distortion. Under this special circumstance, we can ignore the refraction and just deal with a single vector which represents the initial ray vector as it leave/returns to the Mills cross - thus using a virtual transducer will lead to much simpler math, even though a small amount off distortion is unavoidable.

<img align="center" width="80%" style="padding-right:10px;" src="./Images/colocation.PNG">

We will create a virtual transducer at the halfway point between the acoustic center of the transmit transducer at transmit, given by the lever arm `new_la_tx`, and the acoustic center of the receive transducer at reception, given by the lever arm `new_la_rx`

In the LabD.py script add and complete the following:

    # D17.3 Virtual Transducer location; Why colocation?
    new_virtual_trans = (...+...)/2


In [3]:
%run -i LabD.py

print(new_virtual_trans)

NameError: name 'new_virtual_trans' is not defined

    [[ 16.1574]
     [ -1.7485]
     [ 3.6955]]

----
## D17.4 Virtual Transducer location: Vertical Placement

We now know the positions of the transmit and receive transducer in the new reference frame, which coincides with the vessel reference frame as it was at the time of transmit. We however want to position the virtual transducer with respect to the water line, so that we may use its vertical position component as the starting point for our ray tracing. Since we are ray tracing based on a single acoustic center in the virtual array we want to offset its z coordinate by the distance of the rp to the waterline at transmit and receive time. This is the combination of the static draft and average heave (remember that the effects of roll and pitch are already accounted for). We will represent this location by the 3x1 column vector `virtual_trans`.

Remember that the heave is stored in `att_tx[2]` and `att_rx[2]` respectively and that the rp tp waterline offset is given by `vessel.wl`. Make sure to get the signs right!

In the LabD.py script add and complete the following:

    # D17.4 Virtual Transducer location: Vertical Placement
    virtual_trans = new_virtual_trans.copy()
    virtual_trans[2] += (att_tx[3] + att_rx[3]) / 2 - vessel.wl
    
After this we will use $\text{tx}_v$ to indicate the virtual transducer.


In [ ]:
%run -i LabD.py

print(virtual_trans)

    [[ 16.1574]
     [ -1.7485]
     [ 6.3744]]

---
## D18 The Intersection of Non-Orthogonal Cones 

<img align="center" width="80%" style="padding-right:10px;" src="./Images/non_ortho_cones.jpg">

The relative alignment of the transmitter and receiver is precisely known and is usually as close to orthogonal as possible. This is known as it is either machined in the factory for a small sonar or precisely as-placed surveyed after installation on the hull. While their relative alignment is known at a single time the geometry of the Mills Cross used to estimate the formed beam vector cannot assume that same relative misalignment. As we have seen in the previous steps at the time of the receive, the receiver will no longer share the same orientation as the transmitter at time of transmit.

As a result, the beam vector has to modeled by calculating the intersection of non-orthogonal cones

To calculate the resultant beam vector, one must know all of:
    - the orientation of the transmitter long axis at transmit time `t_tx`
    - The orientation of the receiver long axis at receive time `t_rc` 
    - the transit steering angle relative to the long axis of transmit array
    - The receiver steering angle (at the time of receive) relative to the long axis of the receiver array

Each of the steering angles, relative to the long axis of the transmitter and receiver, depend on an assumption of the sound speed at the array. During acquisition, a specific surface sound speed value is used on which the element-specific time delays are based. Thus if there is any reason to reapply a different surface sound speed, all the array-relative steering angles have to be adjusted.

In this section we will be following the cone to cone intersection process as outlined in the document [Hughes Clarke "ccom874_C18_cone_intersection_2016.pdf"](./References/ccom874_C18_cone_intersection_2016.pdf)


---
## D18.0 Ideal Transducer 

Remember that in step D2 we created the unit vectors `tx_ideal` and `rx_ideal`; these are what we will be using in this segment.

Two cones from mutually orthogonal arrays, only intersect if the sum of the two steering angles is less than 90$^{\circ}$. For the case of a multibeam, the transmitter steering is generally less than 5 degrees (up to 15 though if heavy yaw stabilization is utilized). Thus receiver steering angles of up to 85 (75 for a heavy yaw stabilization condition) are allowable.

Remember that the angle between vectors is given by:

$$\cos\alpha = \frac{\overrightarrow{a}\cdot\overrightarrow{b}}{\|a\|\|b\|}$$

Sign Conventions:

Note the conventions used in the following calculations. The unit vector for the transmitter `tx_ideal` is 1,0,0 i.e., it is a vector pointing in the SRF forward or x-axis direction. The unit vector for the the receiver `rx_ideal` is 0,1,0 i.e., a vector pointing in the starboard or positive y-direction. Positive steering is defined as toward the principal vector orientation. Thus +ve forward for transmission, and positive starboard for reception. When unpacking third party sonar telegrams, the sign of the steering must be clarified as it is not a standard. **This differs from the right hand rule for the axis!**

Add the statement `from numpy.linalg import norm` to the imports in your file `LabD.py` 

In [ ]:
%run -i LabD.py

print("\n\ntx_ideal:")
print(tx_ideal)
print("\nrx_ideal:")
print(rx_ideal)

# Thus cos alpha

cos_a = float(tx_ideal.T@rx_ideal)/(norm(tx_ideal)*norm(rx_ideal))
print("\nIdeal angle: %.6f"%(arccos(cos_a)*180/pi))


    tx_ideal:
    [[ 1.0000]
     [ 0.0000]
     [ 0.0000]]

    rx_ideal:
    [[ 0.0000]
     [ 1.0000]
     [ 0.0000]]

    Ideal angle: 90.000000

---
## D18.1 Align Transducer Arrays to the Vessel Reference Frame
 
<img align="center" width="80%" style="padding-right:10px;" src="./Images/align_tx.jpg">

Starting with an ideal transmit vector `tx_ideal` of (1,0,0).
Establishing the as-installed vector accounting for the transmit transducer mounting angles relative to the SRF. **Note that the notation order of angles is reversed in the image above** i.e. the heading is listed first, whereas it is the rotation around the z- or 3- axis. In our code we will keep the order of the angles the same as the order of the axes.

$$\overrightarrow{\text{tx}}_{mount} = \text{R}_z\left(\beta_{tx}\right)\cdot\text{R}_y\left(\phi_{tx}\right)\cdot\text{R}_x\left(\theta_{tx}\right)\cdot\overrightarrow{\text{tx}}_{ideal}$$

$$\overrightarrow{\text{rx}}_{mount} = \text{R}_z\left(\beta_{rx}\right)\cdot\text{R}_y\left(\phi_{rx}\right)\cdot\text{R}_x\left(\theta_{rx}\right)\cdot\overrightarrow{\text{rx}}_{ideal}$$<br>

where:

$\theta_{tx}$: transmit transducer roll mount angle <br>
$\phi_{tx}$: transmit transducer roll mount angle <br>
$\beta_{tx}$: transmit transducer roll mount angle <br><br>
$\theta_{tx}$: receive transducer roll mount angle <br>
$\phi_{tx}$: receive transducer roll mount angle <br>
$\beta_{tx}$: receive transducer roll mount angle <br>

In our code we will use the variable `tx_mount` to represent $\text{tx}_{mount}$ and `rx_mount` to represent $\text{rx}_{mount}$. Remember from D2.0 that the mount angles are given by the arrays `ma_tx` and `ma_rx` respectively 

In the LabD.py script add and complete the following:

    # D18.1 Align Transducer Arrays to the Vessel Reference Frame
    tx_mount = Rz(ma_tx[2])@Ry(...[1])@Rx(...)@tx_ideal
    rx_mount = ...(ma_rx[2])@...(...)@...@rx_ideal



In [ ]:
%run -i LabD.py

print("\n\ntx_mount:")
print(tx_mount)
print("\nrx_mount:")
print(rx_mount)

# Thus cos alpha

cos_a = float(tx_mount.T@rx_mount)/(norm(tx_mount)*norm(rx_mount))
print("\nMount angle: %.6f"%(arccos(cos_a)*180/pi))



    tx_mount:
    [[ 0.9998]
     [ -0.0008]
     [ -0.0179]]

    rx_mount:
    [[ -0.0011]
     [ 1.0000]
     [ 0.0018]]

    Mount angle: 90.108212

---
## D.18.2 Correct for Orientation at Transmit
 
<img align="center" width="80%" style="padding-right:10px;" src="./Images/tx_with_motion.jpg">

Starting with the transmit mount alignment vector `tx_mount` create the transmit vector $\overrightarrow {tx}_{tx}$ for the transmit epoch relative to geo-referenced directions i.e., taking roll, pitch and heading of the SRF into account. We will store the result in the `numpy` array `tx`

$$\overrightarrow{\text{tx}}\left(t_{tx}\right) = \text{R}_z\left(\beta\left(t_{tx}\right)\right)\cdot\text{R}_y\left(\phi\left(t_{tx}\right)\right)\cdot\text{R}_x\left(\theta\left(t_{tx}\right)\right)\cdot\overrightarrow{\text{tx}}_{mount}\Rightarrow$$

$$\overrightarrow{\text{tx}}\left(t_{tx}\right) = \text{R}_z\left(0\right)\cdot\text{R}_y\left(\phi\left(t_{tx}\right)\right)\cdot\text{R}_x\left(\theta\left(t_{tx}\right)\right)\cdot\overrightarrow{\text{tx}}_{mount}$$

where:

$\theta\left(t_{tx}\right)$: roll angle at epoch $t_{tx}$ <br>
$\phi\left(t_{tx}\right)$: pitch angle at epoch $t_{tx}$ <br>
$\beta\left(t_{tx}\right)$: yaw angle in the new reference frame at epoch $t_{tx}$ <br>

Remember that we aligned the new reference frame to the heading i.e., we should use the transmit epoch observed roll and pitch stored in `att_tx`, but zero for the yaw angle.

In the LabD.py script add the following:

    # D.18.2 Correct for Orientation at Transmit
    tx=Rz(0)@Ry(att_tx[1])@Rx(att_tx[0])@tx_mount



In [ ]:
%run -i LabD.py

print("\n\ntx:")
print(tx)



    tx:
    [[ 0.9985]
     [ -0.0011]
     [ -0.0543]]

---
## D.18.3 Correct for Orientation at Receive
 
<img align="center" width="80%" style="padding-right:10px;" src="./Images/rx_with_motion.jpg">

Starting with the receive mount alignment vector `rx_mount` create the receive vector $\overrightarrow {rx}_{rx}$ for the transmit epoch relative to geo-referenced directions i.e., taking roll, pitch and heading of the SRF into account. We will store the result in the `numpy` array `rx`

$$\overrightarrow{\text{rx}}\left(t_{rx}\right) = \text{R}_z\left(\beta\left(t_{rx}\right)\right)\cdot\text{R}_y\left(\phi\left(t_{rx}\right)\right)\cdot\text{R}_x\left(\theta\left(t_{rx}\right)\right)\cdot\overrightarrow{\text{rx}}_{mount}\Rightarrow$$

$$\overrightarrow{\text{rx}}\left(t_{rx}\right) = \text{R}_z\left(\Delta\beta\right)\cdot\text{R}_y\left(\phi\left(t_{rx}\right)\right)\cdot\text{R}_x\left(\theta\left(t_{rx}\right)\right)\cdot\overrightarrow{\text{rx}}_{mount}\Rightarrow$$

where:

$\theta\left(t_{rx}\right)$: roll angle at epoch $t_{rx}$ <br>
$\phi\left(t_{rx}\right)$: pitch angle at epoch $t_{rx}$ <br>
$\beta\left(t_{rx}\right)$: yaw angle in the new reference frame at epoch $t_{rx}$ <br>
$\Delta\beta\left(t_{rx}\right)$: heading difference between epochs  $t_{tx}$ and  $t_{rx}$ $ <br>

Remember that we aligned the new reference frame to the heading at reception i.e., we should use the transmit epoch observed roll and pitch stored in `att_rx`, but `d_h` for the yaw angle.

In the LabD.py script add and complete the following:

    # D.18.3 Correct for Orientation at Transmit
    rx=Rz(...)@Ry(...)@Rx(...)@rx_mount



In [ ]:
%run -i LabD.py

print("\n\nrx:")
print(rx)


    rx:
    [[ -0.0090]
     [ 0.9996]
     [ -0.0258]]

## D18.4 Calculate the Non-Orthogonality angle `non_ortho`

<img align="center" width="80%" style="padding-right:10px;" src="./Images/no.jpg">

$$\|\overrightarrow{\text{tx}}\| = 1, \|\overrightarrow{\text{rx}}\| = 1$$
$$\cos\alpha = \frac{\overrightarrow{a}\cdot\overrightarrow{b}}{\|a\|\|b\|}\Rightarrow$$<br><br>
$$\alpha = \arccos\left(\overrightarrow{\text{tx}}\cdot\overrightarrow{\text{rx}}\right)$$


We expect the angle to be orthogonal i.e. $\pi/2$ so the non-orthogonality is the difference with $\pi/2$.

$$no = \arccos\left(\overrightarrow{\text{tx}}\cdot\overrightarrow{\text{rx}}\right)-\pi/2\Rightarrow$$


$$no = -\arcsin\left(\overrightarrow{\text{tx}}\cdot\overrightarrow{\text{rx}}\right)$$

Note that for the arcsin you need to supply a scalar i.e., if you determine the inner product as `tx.T@rx` you will need to cast it to `float`.

In the LabD.py script add and complete the following:

    # D18.4 Calculate the Non-Orthogonality angle non_ortho
    non_ortho=-arcsin(float(tx.T@rx))
    
also make sure to import the `arcsin` function from `numpy`!

In [ ]:
%run -i LabD.py

print("\n\nnon_ortho:")
print(non_ortho)


    non_ortho:
    0.008630371782925028

## D18.5 Create a New Orthonormal Basis XYZ'

Rather than following steps 5-9 in the document [Hughes Clarke "ccom874_C18_cone_intersection_2016.pdf"](./References/ccom874_C18_cone_intersection_2016.pdf) we will use the more elegant alternative provided in step 5'. Once you get used to **linear algebra** this is a more intuitive and certainly more efficient alternative. 

<img align="center" width="80%" style="padding-right:10px;" src="./Images/no.jpg">

However, to use this solution we will need to create a new Cartesian reference frame $XYZ'$ that has its $XY'$ plane spanning the vectors $\overrightarrow{\text{tx}}$ and $\overrightarrow{\text{rx}}$ - in the figure above this is the blue plane. Cartesian systems are orthonormal and the cross product of two vectors creates a vector orthogonal to them, thus we may use the normalized cross product of $\overrightarrow{\text{tx}}$ and $\overrightarrow{\text{rx}}$ to create a new $Z'$ that is orthogonal to the $XY'$ plane and held by the variable `zp`.

$$\overrightarrow Z' = \frac{\overrightarrow{\text{tx}}\times\overrightarrow{\text{rx}}}{\|\overrightarrow{\text{tx}}\times\overrightarrow{\text{rx}}\|}$$

We have not yet created an orthonormal system. We still need to define the X' axis as variable `xp`, which we may do by simply aligning it to the $\overrightarrow{\text{tx}}$ vector, which is already a unit vector.

$$\overrightarrow{X'} = \overrightarrow{\text{tx}}$$

Finally, we may now define the $Y'$ axes held by the variable `yp`, as it is orthogonal to both $X'$ and $'Y'$ it is simply the normalized cross product:

$$\overrightarrow{Y'} = \frac{\overrightarrow{X'}\times \overrightarrow{Z'}}{\|\overrightarrow{X'}\times \overrightarrow{Z'}\|}$$

$XYZ'$ is now comprised of unit vectors that describe how the transducer array is oriented with respect to the real world we may combine them together in a rotation matrix that establishes the transformation $\text{T}_p$ from the transducer to the real world i.e., if we have a transducer array relative vector $\overrightarrow {bv}$ then the georeferenced version of that vector is $\text{T}_p\overrightarrow {bv}$

$$T_p=\{X'|Y'|Z'\}$$

In the LabD.py script add the following:

    # D18.5 Create a New Orthonormal Basis XYZ'
    xp=tx.copy()
    zp=np.cross(tx.T,rx.T).T
    zp/=norm(zp)
    yp=np.cross(zp.T,xp.T).T
    yp/=norm(yp)
    Tp = np.hstack((xp, yp, zp)) 
    
Note the use of the `numpy` hstack function, which allows us to horizontally augment (combine) the three column vectors to create the Rotation matrix `Tp` defining the coordinate system $XYZ'$

In [ ]:
%run -i LabD.py

print("\n\nTp:")
print(Tp)


---
## D18.6 Formula for Intersection of Orthogonal Arrays
 
<img align="center" width="80%" style="padding-right:10px;" src="./Images/ortho_concentric.jpg">

Let use a unit vector $bv_p$ to represent the beam leaving the transducer array. For now let's consider the ideal case i.e., the arrays are orthogonal. Under this simplest geometry, the resulting beam vector can easily be calculated. Note that the vector is strictly relative to the plane containing the two arrays with the X axis aligned along the transmitter (the $XY'$ plane from D18.5). Due to the magnitude of the beam vector $\overrightarrow{bv_p}$ being one ($\|\overrightarrow{bv}\|=1$), the offset in the $XY$ plane in the $X$-axis direction is $\sin(\text{tx}_\text{steer})$ and in the $Y$-axis is $\sin(\text{rx}_\text{steer})$. The radial distance in this plane is then $\rho_{xy}=\sqrt{\sin(\text{tx}_\text{steer})^2+\sin(\text{rx}_\text{steer})}$ and thus the distance $\rho_{z}$ orthogonal to this plane is $\rho_{z}=\sqrt{1-\rho_{xy}^2}$. Therefore the unit beam vector becomes:

$$ \overrightarrow{\text{bv}_p}=\left[ {\begin{array}{cc}\sin(\text{tx}_\text{steer}) \\\sin(\text{rx}_\text{steer})\\\sqrt{1-\rho_{xy}^2}\\\end{array} } \right]$$


---
### D18.6.0  Formula for Intersection of Non-Orthogonal Arrays
 
<img align="center" width="80%" style="padding-right:10px;" src="./Images/non_ortho_concentric.jpg">

As a small extension of the previous case, if the receiver is oriented at an angle `non_ortho` away from 90 degrees to the transmitter, the formula is slightly different, but still easily calculated.

$$y_1 = \frac{\sin\phi_\text{steer}}{cos(no)}$$<br>
$$y_2 = \sin(\theta_\text{steer})\tan(no)$$<br>

where:

$\theta_\text{steer}$: Transmit steering angle<br>
$\phi_\text{steer}$: Receive steering angle

Given these offsets in the Y' direction we may determine the radial distance $\rho_{hor}$ in the XY' plane as:

$$\rho_{hor}=\sqrt{(y_1+y_2)^2+sin(\theta_\text{steer})^2}$$

Use the steering angles $\phi_\text{steer}$ defined by the variable `-ping.steer_rx[i]` and $\theta_\text{steer}$ defined by the variable `ping.steer_tx[i]`. In the LabD.py script add and complete the following:

    # D18.6.0  Formula for Intersection of Non-Orthogonal Arrays
    y1=sin(...)/cos(...)
    y2=sin(...)*tan(...)
    radial=sqrt((...)**2+sin(...)**2)
    
Note that we have to use the negative of the steering angle provided in the data, due to the use of a different definition of positive beam steering than the one used here.

In [ ]:
%run -i LabD.py

print("\n\ny1:")
print(y1)
print("\ny2:")
print(y2)
print("\nradial:")
print(radial)


    y1:
    0.8437040803510717

    y2:
    -0.0002876541094044535

    radial:
    0.8440747190151341

---
### D18.6.1  Formulate the Beam Vector `bv_p` in XYZ'
 

We have now gotten to the point where we may define the beam vector relative to the transducer, or to be more precise the reference frame XYZ' defined by the transducer arrays.

we will be consistent with the terminology used by Hughes Clarke and name this beam vector `bv_p`, though strictly speaking it is defined in a three dimensional space.

In the new coordinate system the beam vector `bv_p` is given by:

$$ \overrightarrow{\text{bv}}_\text{geo}=\text{T}_p\overrightarrow{\text{bv}}_p$$

In the LabD.py script add and complete the following:
 
    # D18.6.1  Formulate the Beam Vector `bv_p` in XYZ'
    bv_p=np.array([[sin(...)],[...],[sqrt(1-radial**2)]])


In [ ]:
%run -i LabD.py

print("\n\nbv_p:")
print(bv_p)



    bv_p:
    [[ -0.0333]
     [ 0.8434]
     [ 0.5362]]

---
## D18.7 Transform the Beam Vector to Geo Referenced Space
 
Currently the beam vector is transducer array relative, however to be able to apply ray tracing we need to align the beam to a georeferenced space in which the XY plane defines the horizontal and the Z axis defines the vertical. The matrix Tp defined in step D18.5 is the rotation matrix that achieves just this transformation:

$$ \overrightarrow{\text{bv}}_\text{geo} = \text{T}_p\overrightarrow{\text{bv}}_p$$

In our code we will use the variable `bv_g` to represent $\overrightarrow{\text{bv}}_\text{geo}$ 

In the LabD.py script add and complete the following:

    # D18.7 Transform the Beam Vector to Geo Referenced Space
    bv_g=Tp@bv_p 

In [ ]:
%run -i LabD.py

print("\n\nbv_g:")
print(bv_g)


    bv_g:
    [[ -0.0044]
     [ 0.8573]
     [ 0.5149]]

___
## D.18.8 Determine the Depression Angle of the Beam Vector

<img align="center" width="80%" style="padding-right:10px;" src="./Images/theta_az.bmp">

The depression angle $\theta$ can now be determined from the georeferenced beam vector, it is the arctan of its z-component divided by its horizontal component i.e.:

$$\theta_\text{beam} = \arctan\left(\frac{\text{bv}_\text{geo,z}}{\sqrt{\text{bv}^2_\text{geo,x}+\text{bv}^2_\text{geo,y}}}\right)$$

we will represent $\theta_\text{beam}$ by the variable `beam_th`

In the LabD.py script add the following:

    # D.18.8 Determine the Depression Angle of the Beam Vector
    beam_th=float(arctan(bv_g[2]/sqrt(sum(bv_g[0:2]**2))))

In [ ]:
%run -i LabD.py

print("\n\nbeam_th:")
print(beam_th)


    beam_th:
    0.5408497128145111


___
## D18.9 Determine the Azimuth of the Beam Vector

The azimuth is the clockwise angle $\alpha$ from North to the horizontal component of the vector i.e.,

$$\alpha_{hor} = \arctan\left(\frac{bv_{geo,y}}{bv_{geo,x}}\right)$$

where the quadrant needs to be taken into account. Determine the azimuth of the beam and assign it to `beam_az`

In the LabD.py script add the following:

    # D18.9 Determine the Azimuth of the Beam Vector
    beam_az=float(arctan2(...,...))

In [ ]:
%run -i LabD.py

print("\n\nbeam_az:")
print(beam_az)


    beam_az:
    1.575981901668627

___
## D19 At last: Ray Tracing

We now have all the information that we need for the ray tracing, the sound speed at the transducer contained in the variable `ss_tx`, the starting depth `wl_virtual_trans[2]`, the depression angle `beam_th` and the travel time `ping.twtt[i]`

Use the values above with the `SSP` `ray_trace_twtt` method to determine the depth $\text{z}$ represented by the variable `depth` and radial distance $\rho$ represented by the variable `rad_dist` for this beam.

In the LabD.py script add and complete the following:

    # D19 At last: Ray Tracing
    depth,hor_dist,_,_ = ssp.ray_trace_twtt(...,...,...,...)

In [ ]:
%run -i LabD.py

print("\n\ndepth:")
print(depth)
print("\nradial distance:")
print(hor_dist)


    depth:
    [ 97.6591]

    radial distance:
    153.23034710155758

___
## D20 Positioning the Bottom Strike Relative to the Virtual Transducer

Our penultimate step is the determination of where the bottom is relative to the position of the virtual transducer $\overrightarrow{\text{tx}}_v$ that we determined in step D17.4

$$\overrightarrow{\text{bottom}}_\text{vx}=\left[ {\begin{array}{cc}\text{tx}_{v,x}+\rho\cos\alpha \\\text{tx}_{v,y}+\rho\sin\alpha \\\text{z}\\\end{array} } \right]$$

In the LabD.py script add and complete the following:

    # D20 Positioning the Bottom Relative to the Virtual Transducer
    bot_vx = np.zeros((3,1))
    bot_vx[0] = virtual_trans[0]+hor_dist*cos(beam_az)
    bot_vx[1] = virtual_trans[...]+...*sin(...)
    bot_vx[2] = depth    

In [ ]:
%run -i LabD.py

print("\n\nbottom relative to the virtual array:")
print(bot_vx)



    bottom relative to the virtual array:
    [[ 15.3628]
     [ 151.4798]
     [ 97.6591]]

___
## D21 Positioning the Bottom Strike Relative to the RP at Transmit

Our final step is the determination of where the bottom is relative to the position of the reference point rp at the time of transmission. To do this we can simply offset the virtual transducer horizontal coordinates by the virtual transducer location 

In the LabD.py script add and complete the following:

    # D21 Positioning the Bottom Strike Relative to the RP at Transmit
    bot_rp_tx = bot_vx.copy()
    bot_rp_tx[0] -= virtual_trans[0]
    bot_rp_tx[1] -= virtual_trans[1]
      

In [ ]:
%run -i LabD.py

print("\n\nbottom relative to the rp:")
print(bot_rp_tx)



___

Complete the following questions and submit the answers in a **WORD** document (please no *PDF* or other formats). If you do not have access to word please contact me first.

Keep your answers concise, but they should consist of at least one full sentence.

    1. How is the mismatch between the sound speed at the transducer addressed?
    2. You would need to alter for use in the polar regions, why?
        hint - this has to do with the positioning
    3. Explain why the crab angle is not necessarily the negative value of the drift angle
    4. Take the figure of D16 and show the drift angle associated to geo_pos_rp_rx
    5. Even if the transducers are mounted perfectly orthogonal we typically still will find a non-orthogonality in our calculation of step D18.4, why is that?

